### Note on TRITON DEPLETION options 


- The power/flux provided in the BURNDATA block is mainly used for scaling the transport flux solution to the actual flux/power level of the system
-  Since in some situations power/flux can be known for only selected materials in the system, TRITON has option to enable the user to specify power/flux for those materials by putting negative sign in front of them in the DEPLETION block
- TRITON approach in case power if given:
    - initialize variable (e.g., power_unnormalized) as zero
    - loop over all materials, get thier unnormalized (1 group collpased) flux form the transport, and compute corresponding power based on 1-group cross secions (i.e., fission, capture) and on material amount (i.e., N)[[code](https://code-int.ornl.gov/rnsd/scale/-/blob/master/packages/Origen/Core/dc/Material.cpp?ref_type=heads#L1824)]
        
        For material $i$, the power is computed as
        $$

           p_i = \kappa_f \times N{_i}\times\sigma_f{_i} \times \phi_{i_{\text{unnormalized}}} + \kappa_c \times N{_i}\times\sigma_c{_i}  \times \phi_{{i}_{\text{unnormalized}}}

        $$

    - accumulate the material power in the variable power_unnormalized:

       $$
        \text{power\_unnormalized} = \sum_{i}^{m}  p{_i}
       $$
    - now compute global scaling factor $f$ as [[code](https://code-int.ornl.gov/rnsd/scale/-/blob/master/packages/Module/TDepl1d/OrigenDepletion.cpp#L983)]
    $$
       f = \frac{\text{power\_input}}{\text{power\text{\_unnormalized}}}
    $$

    - use this scaling factor to scale all materials fluxes and powers (e.g., $p_i$, $\phi_{i_\text{unnormalized}}$). [[code](https://code-int.ornl.gov/rnsd/scale/-/blob/master/packages/Module/TDepl1d/OrigenDepletion.cpp#L1001)]
    
    - note that if there are not specific materials used for as basis for normlization, we use **powers** of all materials to compute the scaling factor

Rabab's changes in # 1839:

- What if the user input flux level not power?
    -  In that case, we accumulate unnormalized fluxes for those materials whose their flux is used as basis of normalization (with negative sign before them)

    $$
    \phi_\text{total} = \sum_{i=1}^{m} \phi_{{i}_\text{{unnormalized}}}
    $$

    - the scaling factor f is computed as 
        $$
        f = \frac{\text{flux\_input}}{\phi_\text{total}}
        $$

    - use this scaling factor to scale all materials fluxes and powers (e.g., $p_i$, $\phi_{i_\text{unnormalized}}$).
    - note that if there are not specific materials used for normlization, we use **fluxes** of all materials to compute the scaing factor




### ORIGEN depletin options

At this point, we have scaled/normalized flux and power for each material based on user information and transport solution of the flux. Now, how ORGIEN will use these information?    

There are two modes of depletion in ORIGEN calcualtions:    
- Deplete by power (defalut), where the power over ORIGEN depletion interval is assumed constant.   
  - as the material inventory changes over sub-intervals, the flux will be re-computed to keep the power constant    
  - In this mode, the scaled power of the material is passed to ORIGEN   
  - [[code](https://code-int.ornl.gov/rnsd/scale/-/blob/master/packages/Module/TDepl1d/OrigenDepletion.cpp#L1018)]

- Deplete by flux (material the are preceded by **flux** keyword in the depletion block)   
   - the **flux** over ORIGEN depletion interval is assumed to be constant. So as the fuel inventory change over sub-intervals, the power will be adjusted accordingly, however the flux will remain constant
   - In this mode, the scaled flux of the material is passed to ORIGEN
   - [[Code]( https://code-int.ornl.gov/rnsd/scale/-/blob/master/packages/Module/TDepl1d/OrigenDepletion.cpp#L1005)]